## Introduction

I will list and visualize all major parts of New York City that has great mexican resturants.

For this project we need the following data:
New York City data that contains list Boroughs, Neighborhoods along with their latitude and longitude.
Mexican resturants in each neighborhood of new york city.
GeoSpace data

Questions that will be answers at the end of this project :
1. What are best location in New York City for Mexican Food?
2. Which areas have potential Mexican Restaurant Market?
3. Which all areas lack Mexican Restaurants?
4. Which is the best place to stay if you prefer Mexican Food?

## Collecting Data

The data used bla bla bla

In [7]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

import numpy as np # library to handle data in a vectorized manner
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from geopy.geocoders import ArcGIS # module to convert an address into latitude and longitude values
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

from sklearn.cluster import KMeans# k-means for categorization

In [8]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude

In [9]:
def get_venues(lat,lng):
    
    #set variables
    radius=1000
    LIMIT=100
    CLIENT_ID = 'SISDD1LUOONMBRCNBYLCXQVNFS3SVY4HGO3WOKDVB3L0IAQ4'
    CLIENT_SECRET = 'YTSCOGI1Y0LWYZ5U3F5MCRHVXWLB4FEA43UFOOGTHZUSBAWG' 
    VERSION = '20180605' 
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

In [10]:
def get_venue_details(venue_id):
        
    CLIENT_ID = 'SISDD1LUOONMBRCNBYLCXQVNFS3SVY4HGO3WOKDVB3L0IAQ4'
    CLIENT_SECRET = 'YTSCOGI1Y0LWYZ5U3F5MCRHVXWLB4FEA43UFOOGTHZUSBAWG' 
    VERSION = '20180605'
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

In [11]:
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return new_york_data

In [12]:
new_york_data=get_new_york_data()

In [13]:
new_york_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [14]:
# prepare neighborhood list that contains Mexican resturants
column_names=['Borough', 'Neighborhood', 'ID','Name']
mexican_rest_ny=pd.DataFrame(columns=column_names)
count=1
for row in new_york_data.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    mexican_resturants=venues[venues['Category']=='Mexican Restaurant']   
    print('(',count,'/',len(new_york_data),')','Mexican Resturants in '+Neighborhood+', '+Borough+':'+str(len(mexican_resturants)))
    for resturant_detail in mexican_resturants.values.tolist():
        id, name , category=resturant_detail
        mexican_rest_ny = mexican_rest_ny.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1

( 1 / 306 ) Mexican Resturants in Wakefield, Bronx:0
( 2 / 306 ) Mexican Resturants in Co-op City, Bronx:2
( 3 / 306 ) Mexican Resturants in Eastchester, Bronx:0
( 4 / 306 ) Mexican Resturants in Fieldston, Bronx:3
( 5 / 306 ) Mexican Resturants in Riverdale, Bronx:2
( 6 / 306 ) Mexican Resturants in Kingsbridge, Bronx:6
( 7 / 306 ) Mexican Resturants in Marble Hill, Manhattan:3
( 8 / 306 ) Mexican Resturants in Woodlawn, Bronx:0
( 9 / 306 ) Mexican Resturants in Norwood, Bronx:2
( 10 / 306 ) Mexican Resturants in Williamsbridge, Bronx:0
( 11 / 306 ) Mexican Resturants in Baychester, Bronx:2
( 12 / 306 ) Mexican Resturants in Pelham Parkway, Bronx:0
( 13 / 306 ) Mexican Resturants in City Island, Bronx:0
( 14 / 306 ) Mexican Resturants in Bedford Park, Bronx:2
( 15 / 306 ) Mexican Resturants in University Heights, Bronx:1
( 16 / 306 ) Mexican Resturants in Morris Heights, Bronx:1
( 17 / 306 ) Mexican Resturants in Fordham, Bronx:2
( 18 / 306 ) Mexican Resturants in East Tremont, Bronx:

( 141 / 306 ) Mexican Resturants in Sunnyside, Queens:4
( 142 / 306 ) Mexican Resturants in East Elmhurst, Queens:2
( 143 / 306 ) Mexican Resturants in Maspeth, Queens:0
( 144 / 306 ) Mexican Resturants in Ridgewood, Queens:5
( 145 / 306 ) Mexican Resturants in Glendale, Queens:0
( 146 / 306 ) Mexican Resturants in Rego Park, Queens:2
( 147 / 306 ) Mexican Resturants in Woodhaven, Queens:1
( 148 / 306 ) Mexican Resturants in Ozone Park, Queens:1
( 149 / 306 ) Mexican Resturants in South Ozone Park, Queens:0
( 150 / 306 ) Mexican Resturants in College Point, Queens:1
( 151 / 306 ) Mexican Resturants in Whitestone, Queens:0
( 152 / 306 ) Mexican Resturants in Bayside, Queens:2
( 153 / 306 ) Mexican Resturants in Auburndale, Queens:1
( 154 / 306 ) Mexican Resturants in Little Neck, Queens:0
( 155 / 306 ) Mexican Resturants in Douglaston, Queens:0
( 156 / 306 ) Mexican Resturants in Glen Oaks, Queens:0
( 157 / 306 ) Mexican Resturants in Bellerose, Queens:0
( 158 / 306 ) Mexican Resturants

( 276 / 306 ) Mexican Resturants in Stuyvesant Town, Manhattan:2
( 277 / 306 ) Mexican Resturants in Flatiron, Manhattan:1
( 278 / 306 ) Mexican Resturants in Sunnyside Gardens, Queens:6
( 279 / 306 ) Mexican Resturants in Blissville, Queens:0
( 280 / 306 ) Mexican Resturants in Fulton Ferry, Brooklyn:1
( 281 / 306 ) Mexican Resturants in Vinegar Hill, Brooklyn:1
( 282 / 306 ) Mexican Resturants in Weeksville, Brooklyn:0
( 283 / 306 ) Mexican Resturants in Broadway Junction, Brooklyn:2
( 284 / 306 ) Mexican Resturants in Dumbo, Brooklyn:1
( 285 / 306 ) Mexican Resturants in Manor Heights, Staten Island:0
( 286 / 306 ) Mexican Resturants in Willowbrook, Staten Island:1
( 287 / 306 ) Mexican Resturants in Sandy Ground, Staten Island:1
( 288 / 306 ) Mexican Resturants in Egbertville, Staten Island:0
( 289 / 306 ) Mexican Resturants in Roxbury, Queens:0
( 290 / 306 ) Mexican Resturants in Homecrest, Brooklyn:2
( 291 / 306 ) Mexican Resturants in Middle Village, Queens:0
( 292 / 306 ) Mexic